In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from pandas import ExcelWriter
import math
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import f1_score
import os
from keras import regularizers

from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from matplotlib import pyplot



os.chdir("C:/Users/VIP13/Статья 2/Котировки 6")

In [ ]:
# split a univariate dataset into train/test sets
# forecast monthly births with xgboost
 
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test, :], data[-n_test:, :]
 
# fit an xgboost model and make a one step prediction
def xgboost_forecast(train, testX,eta, gamma, max_depth):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = XGBRegressor(objective='reg:squarederror', n_estimators=100, eta=eta, gamma=gamma, max_depth=max_depth)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict(asarray([testX]))
	return yhat[0]
 
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, eta, gamma, max_depth):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1], test[i, -1]
		# fit model on history and make a prediction
		yhat = xgboost_forecast(history, testX, eta=eta, gamma=gamma, max_depth=max_depth)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
		# summarize progress
# 		print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
	# estimate prediction error
	error = mean_absolute_error(test[:, -1], predictions)
	return error, test[:, -1], predictions

def progon(frame, n_in, n_test, eta, gamma, max_depth):
    iteratation_5=int((len(frame)-399)/5)+1
    y=[]
    yhat=[]

    for i in range(0,iteratation_5):
        series=DataFrame(np.log(np.sqrt(frame.iloc[0+5*i:399+5*i+1])))
        values = series.values
        # transform the time series data into supervised learning
        data = series_to_supervised(values, n_in=5)
        # evaluate
        progn=walk_forward_validation(data, n_test=1, eta=eta, gamma=gamma, max_depth=max_depth)

        y.append(progn[1])
        yhat.append(progn[2])
    c=np.exp(y)
    cc=np.exp(yhat)
    mape=(100*abs(c-cc)/c).mean()
    
    
    frame_pr=pd.DataFrame(c)
    frame_pr['prog']=cc
    fr=frame_pr.iloc[1:,0:].copy()
    ind=[i for i in range(0,len(frame_pr)-1)]
    fr.index=ind
    frr=frame_pr.iloc[:-1,0:].copy()
    frr.index=ind

    frame_diff=fr-frr
    frame_diff[frame_diff < 0] = 0
    frame_diff[frame_diff > 0] = 1
    f1=[]
    f1.append(0)
    accuracy=[]
    for i in range(1,len(frame_diff.columns)):
        accuracy.append(accuracy_score(frame_diff.iloc[:,0], frame_diff.iloc[:,i]))

    return c, cc, mape, accuracy[0]

In [ ]:
names=[]
for eta in np.arange(0.1,1,0.1):
    for gamma in np.arange(0.1,1,0.1):
        for max_depth in np.arange(1,11,1):
#             for reg_lambda in np.arange(0,1,0.1):
#                 for reg_alpha in np.arange(0,1,0.1):
#                     for subsample in np.arange(0,1,0.1):
#                         for colsample_bytree in np.arange(0,1,0.1):
            name='boost'+'_'+str(eta)+'_'+str(gamma)+'_'+str(max_depth)
            names.append(name)

all_frame=pd.DataFrame(columns=names)                            


In [ ]:
frame=pd.read_csv('all_snp_btc_rv_rr.csv', delimiter=';',decimal='.')

fraa=pd.DataFrame()

fraa['y']=frame['rv_shk_snp']
# fraa['y']=frame['rv_shk_btc']

ll=0
mapes=[]
accuracys=[]
for eta in np.arange(0.1,1,0.1):
    for gamma in np.arange(0.1,1,0.1):
        for max_depth in np.arange(1,11,1):
            ll=ll+1
            name='boost'+'_'+str(eta)+'_'+str(gamma)+'_'+str(max_depth)
            prog=progon(fraa, 5, 1, eta, gamma, max_depth)
            
            pr=[]
            for i in prog[1]:
                pr.append(i)

            all_frame[name]=pr
#             all_frame[name]=np.array(prog[1])
            mapes.append(prog[2])
            accuracys.append(prog[3])
            print(ll)
            
            
pr=[]
for i in prog[0]:
    pr.append(i)
all_frame['orig_data']=pr
all_frame.to_csv('all_frame_snp_boosting.csv')
# all_frame.to_csv('all_frame_btc_boosting.csv')

In [ ]:
frame=pd.read_csv('all_frame_snp_boosting.csv', delimiter=',',decimal='.')
# frame=pd.read_csv('all_frame_btc_boosting.csv', delimiter=',',decimal='.')
orig=frame['orig_data']

# frame.insert(0, "exp_prog", np.array(frame1['exp_prog']))
# frame['exp_prog']=frame1['exp_prog']
origg=[]
pri=[]
for i in orig:
    origg.append(float(i[1:-1]))
frae=pd.DataFrame(origg)

for i in range(1, len(frame.columns)-1):
    prognoz=frame[frame.columns[i]]
    pri=[]
    for j in prognoz:
        pri.append(float(j[1:-1]))
    frae[frame.columns[i]]=pri  
        
mape=[]
mape.append(0)
c=frae[frae.columns[0]]
for i in range(1,len(frae.columns)):
    cc=frae[frae.columns[i]]
    mape.append((100*abs(c-cc)/c).mean())
# print(mape)
fraes=frae.transpose()
fraes['mape']=mape
frae_sort=fraes.sort_values(by='mape')
frae_sort=frae_sort.transpose()
# print(frae_sort)
print(frae_sort.iloc[-1,1:6])
frame_pr=pd.DataFrame(frae[frae.columns[0]])
accuracy=[]
accuracy.append(0)
for i in range(1,len(frae.columns)):
    frame_pr['prog']=frae[frae.columns[i]]
    fr=frame_pr.iloc[1:,0:].copy()
    ind=[i for i in range(0,len(frame_pr)-1)]
    fr.index=ind
    frr=frame_pr.iloc[:-1,0:].copy()
    frr.index=ind
    frame_diff=fr-frr
    frame_diff[frame_diff < 0] = 0
    frame_diff[frame_diff > 0] = 1

    for j in range(1,len(frame_diff.columns)):
        accuracy.append(accuracy_score(frame_diff.iloc[:,0], frame_diff.iloc[:,j]))
# print(accuracy)
fraes['accuracy']=accuracy

frae_sort_1=fraes.sort_values(by='accuracy', ascending=False)
frae_sort_1=frae_sort_1.transpose()
print(frae_sort_1.iloc[-1,0:5])